# XGBoost

In [ ]:
import xgboost

In [ ]:
#!pip install xgboost

In [ ]:
# 관련 파일 임포트

from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score

from sklearn.datasets import load_breast_cancer

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

## 위스콘신 유방암 데이터셋

In [ ]:
#데이터 준비

In [ ]:
b_cancer = ??


#b_cancer를 데이터프레임으로 만드시오(cancer_df)
# 마지막열을 "target"으로 명명하고 결과값을 할당한다.

cancer_df = ??
cancer_df["target"] = ??

X = ??
y = ??


# 데이터 분할
# test_size=0.2, random_state=156


In [ ]:
### 1. 조기종료 옵션을 사용하지 않는 경우

In [ ]:
### 2. 조기 종료 옵션을 사용한 경우

In [ ]:
* 하이퍼파라미터 튜닝( 조기종료 옵션 사용)

In [ ]:
# 조기종료 모델에 대한 평가(early_stopping_rounds:100)



In [ ]:
# 조기종료 모델 구축 및 평가(early_stopping_rounds=10)

* 시각화

In [ ]:
# 1)  xgboost가 제공하는 Plot_importance



# LightGBM(LGBM)

In [ ]:
import lightgbm

In [ ]:
#!pip install lightgbm

## 위스콘신 유방암 데이터셋

In [ ]:
# 조기 중단 옵션을 사용하지 않은 경우

In [ ]:
# 조기중단 옵션을 사용한 경우

In [ ]:
# 시각화

## CatBoosting